In [1]:
using Distributions
using JuMP
using CPLEX

7

In [ ]:
# Demand distribution
mu = [100.0, 200, 150, 170, 180, 170, 170]
sigma = [20.0, 50, 30, 50, 40, 30, 50]

# For each location, we discretize the levels into
# low, medium, high, using quantiles.
q = [
    quantile(Normal(mu[i], sigma[i]), [1/6, 3/6, 5/6])
    for i in 1:7
]

const d = vec(collect(Iterators.product(q...)))

# Number of locations
const N = 7

In [2]:
# Solve a subproblem given first stage decision s and demand realization d,
# and generate cut
function benders_cut(s, d)
    # holding cost 
    h = 1.0
    # shortage cost
    p = 4.0
    # transshipment cost
    c = 0.5

    # Sanity check
    @assert(length(s) == N)
    @assert(length(d) == N)

    # Build second stage model
    model = Model(CPLEX.Optimizer)

    # Suppress output for solving subproblems
    # to avoid cluttering the output
    set_silent(model)

    ij_set = [(i, j) for i=1:N for j=1:N if i!=j]
    @variables(model, begin
        e[1:N] >= 0
        f[1:N] >= 0
        q[1:N] >= 0
        r[1:N] >= 0
        t[ij_set] >= 0
    end)

    @objective(model, Min, h*sum(e) + c*sum(t) + p*sum(r))

    @constraints(model, begin
        B[i=1:N], f[i] + sum(t[(i,j)] for j=1:N if j!=i) + e[i] == s[i]
        M[i=1:N], f[i] + sum(t[(j,i)] for j=1:N if j!=i) + r[i] == d[i]
        R, sum(r) + sum(q) == sum(d)
        E[i=1:N], e[i] + q[i] == s[i]
    end)

    optimize!(model)

    # Get dual variables corresponding to that constraint
    pi_B = dual.(B)
    pi_M = dual.(M)
    pi_R = dual(R)
    pi_E = dual.(E)

    # Cut coefficients
    alpha = sum(d[i] * (pi_M[i] + pi_R) for i=1:N)
    beta = pi_B + pi_E
    
    obj = objective_value(model)
    dual_obj = alpha + beta'*s
    @assert(isapprox(obj, dual_obj), "obj=$obj, dual_obj=$dual_obj")
    
    return alpha, beta, dual_obj
end


benders_cut (generic function with 1 method)

In [3]:
# Given first stage decision s, go through each scenario
# to get an aggregated Benders cut.
function get_aggregate_cuts(s)
    a::Float64 = 0.0
    b::Vector{Float64} = zeros(N)
    d_obj::Float64 = 0.0

    for i in eachindex(d)
        alpha, beta, dual_obj = benders_cut(s, d[i])
        a += alpha / length(d)
        b += beta / length(d)
        d_obj += dual_obj / length(d)
    end

    return a, b, d_obj
end

get_aggregate_cuts (generic function with 1 method)

In [4]:
# Master problem
model = Model(CPLEX.Optimizer)

# Some initial bounds on s
@variable(model, 80 <= s[1:N] <= 300)
@variable(model, eta >= 0)
@objective(model, Min, eta)
# Suppress output
set_silent(model)
optimize!(model)

# TODO: Print out solutions, objective value and number of iterations

In [5]:
# Maximum number of iterations
MAX_ITER = 100

# Stopping criteria
ABSOLUTE_OPTIMALITY_GAP =  1e-6
@time begin
    for iter = 1:MAX_ITER
        # ====================
        # Hint:
        # Solve master problem to get lower_bound and new first stage decision.
        # Call get_aggregate_cuts on current first stage decision
        # to get alpha, beta, and upper_bound.
        # Add a new cut to the master problem.
        # TODO: fill in
        optimize!(model)
        lower_bound = objective_value(model)
        cut = get_aggregate_cuts(value.(s))
        alpha = cut[1]
        beta = cut[2]
        upper_bound = cut[3]

        gap = (upper_bound - lower_bound)
        println(iter)

        # =====================
        # Recommended: Display upper_bound and lower_bound
        @show value.(lower_bound)
        @show value.(upper_bound)

        @show value.(s)
        @show objective_value(model)

        # =====================
        # Stopping criteria
        # TODO: fill in

        new_cut = @constraint(model,eta>=alpha + beta'*s)

        if gap < ABSOLUTE_OPTIMALITY_GAP
            println("Terminating with the optimal solution")
            break
        end
    end
end

1
value.(lower_bound) = 0.0
value.(upper_bound) = 2320.0
value.(s) = [80.0, 80.0, 80.0, 80.0, 80.0, 80.0, 80.0]
objective_value(model) = 0.0
2
value.(lower_bound) = 0.0
value.(upper_bound) = 337.418661144916
value.(s) = [300.0, 300.0, 219.99999999996209, 80.0, 80.0, 80.0, 80.0]
objective_value(model) = 0.0
3
value.(lower_bound) = 0.0
value.(upper_bound) = 364.65373072721803
value.(s) = [300.0, 300.0, 300.0, 224.5109783554474, 80.0, 80.0, 80.0]
objective_value(model) = 0.0
4
value.(lower_bound) = 0.0
value.(upper_bound) = 266.05085698939115
value.(s) = [80.0, 80.0, 287.55802396597926, 80.0, 300.0, 300.0, 102.38330913087796]
objective_value(model) = 0.0
5
value.(lower_bound) = 10.012999785927803
value.(upper_bound) = 263.9626343319121
value.(s) = [137.37569020450104, 80.0, 80.0, 300.0, 80.0, 201.94064289235573, 300.0]
objective_value(model) = 10.012999785927803
6
value.(lower_bound) = 36.02982201346198
value.(upper_bound) = 232.71671695527746
value.(s) = [80.0, 300.0, 80.0, 129.764053965

34
value.(lower_bound) = 143.21232071415736
value.(upper_bound) = 150.13482361509307
value.(s) = [105.45862533709337, 211.41841020521883, 179.14489997704604, 197.37699340535346, 191.41841020522133, 197.25474041136835, 157.73213556526707]
objective_value(model) = 143.21232071415736
35
value.(lower_bound) = 143.40381457033897
value.(upper_bound) = 147.13561398103914
value.(s) = [108.57997519285392, 205.96083530404394, 172.36601746821756, 185.73416694951445, 185.96083530404587, 192.3907964643571, 185.70938795339578]
objective_value(model) = 143.40381457033897
36
value.(lower_bound) = 144.28534199019566
value.(upper_bound) = 154.58695587100408
value.(s) = [122.948872136528, 218.22210823236625, 170.82443807619705, 179.38205503765474, 160.8373038381183, 217.2447552154381, 152.96173789844852]
objective_value(model) = 144.28534199019566
37
value.(lower_bound) = 144.28534199019617
value.(upper_bound) = 152.1190946259371
value.(s) = [92.0440304940381, 218.22210823236696, 170.82443807620385, 179.